In [1]:
import pandas as pd
from ipynb.fs.full.semrush_call import *

In [37]:
def get_authority_score_websites(dataframe, linktype):
    # Check what type of backlink has to be called
    if linktype == 'domain_ascore':
        type_params = 'backlinks_refdomains'
        target_type = 'root_domain'
        export_columns = 'domain_ascore'
    elif linktype == 'page_ascore':
        type_params = 'backlinks'
        target_type = 'root_domain'
        export_columns = 'page_ascore'
    elif linktype == 'ascore':
        type_params = 'backlinks_competitors'
        target_type = 'root_domain'
        export_columns = 'ascore'
    else:
        print("The entered parameters are invalid.")
    
    # Define SEMrush call params
    backlink_params = {
    '?key': 'b9fd485dbbcc0d31bc30e828806fd14c',
    'type': type_params,
    'target': '',
    'target_type': target_type,
    'export_columns': export_columns,
    'display_limit': '1'
    }
    endpoint_url = 'https://api.semrush.com/analytics/v1/'
    
    # Create the list that gets returned at the end
    listje = []
    
    # Make a copy of the dataframe
    finalBacklinkData = dataframe.copy()
    
    finalBacklinkData[export_columns] = 0
        
    for i in range(0, (len(finalBacklinkData))-4):
        try:
            backlink_params['target'] = finalBacklinkData.Dn[i]
            data = requests.get(semrush_call(backlink_params, endpoint_url))
            parsed_data = parse_response(data.content)
            print(parsed_data)
        
        
            conv_parsed_data = str(parsed_data)
            print(conv_parsed_data)
            conv_parsed_data = conv_parsed_data.split("'")
            print(conv_parsed_data)
            conv_parsed_data = int(conv_parsed_data[1])
            print(conv_parsed_data)
            listje.append(conv_parsed_data)
        except:
            listje.append(None)
                
    return listje

In [35]:
dftest = pd.read_csv('calls/dataframe-437c05d23965ee61d3a21893f2c298f06909cd0675ace4caa20fd8b77fc8216b.csv')
dftest = dftest.head()

In [39]:
lol = get_authority_score_websites(dftest, 'ascore')
lol

[{0: '50'}]
[{0: '50'}]
['[{0: ', '50', '}]']
50


[50]

In [ ]:
# finalBacklinkData = backlink(dftest, 'backlink', True)
# finalBacklinkData

In [ ]:
def get_authority_score_websites(dataframe, linktype, potential_overwrite):
    # Check what type of backlink has to be called
    if linktype == 'domain_ascore':
        type_params = 'backlinks_refdomains'
        target_type = 'root_domain'
        export_columns = 'domain_ascore'
    elif linktype == 'page_ascore':
        type_params = 'backlinks'
        target_type = 'root_domain'
        export_columns = 'page_ascore'
    elif linktype == 'ascore':
        type_params = 'backlinks_competitors'
        target_type = 'root_domain'
        export_columns = 'ascore'
    else:
        print("The entered parameters are invalid.")
    
    # Define SEMrush call params
    backlink_params = {
    '?key': 'b9fd485dbbcc0d31bc30e828806fd14c',
    'type': type_params,
    'target': '',
    'target_type': target_type,
    'export_columns': export_columns,
    'display_limit': '1'
    }
    endpoint_url = 'https://api.semrush.com/analytics/v1/'
    
    # Create the list that gets returned at the end
    backlinklist = []
    
    # Make a copy of the dataframe
    finalBacklinkData = dataframe.copy()

    # Hash the given 'Ur' values, save it and check if the file already exists
    backlinklist_hashed = hash_variable(finalBacklinkData['Ur']) + linktype
    file_exists = existing_file(backlinklist_hashed, "backlinklist", potential_overwrite)
    
    # Check if the file exists and act accordingly
    if file_exists == False:    
        finalBacklinkData[export_columns] = 0
    
        for i in range(0, (len(finalBacklinkData))):
            try:
                # Check what type of data has to be retrieved from SEMrush
                if linktype == 'backlink':
                    backlink_params['target'] = finalBacklinkData.Dn[i]
                elif linktype == 'internal' or linktype == 'external':
                    backlink_params['target'] = finalBacklinkData.Ur[i]

                data = requests.get(semrush_call(backlink_params, endpoint_url))
                parsed_data = parse_response(data.content)
        
                conv_parsed_data = str(parsed_data)
                conv_parsed_data = conv_parsed_data.split("'")
                conv_parsed_data = int(conv_parsed_data[1])
                backlinklist.append(conv_parsed_data)
            except:
                backlinklist.append(None)
                
        # Convert list to dataframe and save it
        backlinklist_pd = pd.DataFrame(backlinklist)
        backlinklist_pd.to_csv(f"calls/backlinklist-{backlinklist_hashed}.csv")
        return backlinklist
    
    else:
        # Load CSV file and convert back to list
        backlinklist_pd = pd.read_csv(f"calls/backlinklist-{backlinklist_hashed}.csv")
        return backlinklist_pd['0'].values.tolist()